# ARIMA - SARIMA

Não vou utilizar ARIMA ou SARIMA para fazer previsões sérias, é mais para aprender algumas técnicas e como separar os dataset de teste e treino.

# Ìndice

1. [Resumo e objetivos](#Resumo-e-objetivos)
- [Série temporal básica](#Série-temporal-básica)
    - [O que é estacionaridade?](#O-que-é-estacionaridade?)
    - [Checando estacionaridade](#Checando-estacionaridade)
    - [Transformando em uma série estacionária](#Transformando-em-uma-série-estacionária)
    - [Separando o dataset](#Separando-o-dataset)
- [Série temporal](#Série-temporal)
    - [ARIMA](#ARIMA)
    - [SARIMA](#SARIMA)
- [Apéndice](#Apéndice) 
    - [O que é a hipótese nula e como é utilizado no ADF teste](#null)
    - [O andar do bêbado](#O-andar-do-bêbado)
- [Referências](#Referências)

Explicar:
- [ ] Escolher uma loja para ser estudada. (Olhar na EDA)
- [ ] O que é estacionaridade de um série temporal.
- [ ] Tipos de estacionaridade. (Strict Stationary, Trend Stationary, Difference Stationary)
- [ ] Como checar a estacionaridade.
- [ ] Como fazer a série estacionária.

- [A Gentle Introduction to Handling a Non-Stationary Time Series in Python](https://www.analyticsvidhya.com/blog/2018/09/non-stationary-time-series-python/)

# Resumo e objetivos

Começar com:
- https://www.analyticsvidhya.com/blog/2018/09/non-stationary-time-series-python/
- https://www.analyticsvidhya.com/blog/2018/02/time-series-forecasting-methods/
- https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/
- https://www.analyticsvidhya.com/blog/2018/10/predicting-stock-price-machine-learningnd-deep-learning-techniques-python/

# Carregando os arquivos

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.stattools import adfuller

from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import missingno as msno

path = 'data'

train = pd.read_csv(path + '/sales_train.csv.gz')
test = pd.read_csv(path + '/test.csv.gz').set_index('ID')
items = pd.read_csv(path + '/items.csv')
items_cat = pd.read_csv(path + '/item_categories.csv')
shops = pd.read_csv(path + '/shops.csv')
geo = pd.read_csv(path + '/geo_shop.csv')

# Série temporal básica

- [O que é estacionaridade?](#O-que-é-estacionaridade?)
- [Checando estacionaridade](#Checando-estacionaridade)
- [Transformando em uma série estacionária](#Transformando-em-uma-série-estacionária)
- [Separando o dataset](#Separando-o-dataset)
- [Considerações finais](#Considerações-finais)

### O que é estacionaridade?

[Artigo básico][1] [Artigo complexo][2] [ML][3]

[1]: https://www.analyticsvidhya.com/blog/2018/09/non-stationary-time-series-python/
[2]: https://www.quantstart.com/articles/Serial-Correlation-in-Time-Series-Analysis
[3]: https://machinelearningmastery.com/time-series-data-stationary-python/

Uma série é considerada estacionária quando sua média, variância e convariância não mudam com o tempo. As séries abaixo não são estacionárias,

![non-stationary](non-stat.png)

- A primeira exibe uma tendência de subida.
- Na segunda sua variância muda com o tempo. De forma mais simples, seu valor máximo e minímo ficam variando muito com o tempo.
- A terceira exibe uma covariância com o tempo. De forma simples, o espaçamento entre os picos e vales deveriam ser iguais.

Então o que de fato queremos é uma série perfeitinha, tipo essa,

![non-stationary](stat.png)

O que nunca vai acontecer no mundo real. Existem técnicas que transformam séries não estacionárias em estacionárias, algumas delas serão vistas nesse notebook. Para enteder mais da parte técnica recomendo esse [site][2], mas é bom já ter uma base matemática boa antes de ler. Esse [site][1] é mais simples e também muito bom, inclusive tirei as figuras de lá.

[1]: https://www.analyticsvidhya.com/blog/2018/09/non-stationary-time-series-python/
[2]: https://www.quantstart.com/articles/Serial-Correlation-in-Time-Series-Analysis

### Checando estacionaridade

Nem sempre vai ser possível checar no olho se a série é estacionária ou não. Precisamos utilizar alguns testes matemáticos para isso, então vamos pegar uma loja do dataset e fazer algumas analizes. 

> Esse dataset não é muito bom para fazer esse tipo de coisa, ele é muito simples. Eu deixei nas referências dois kerneis com dataset melhores para isso.

Vou unir o dataset todo, agrupar tudo por mês e escolher uma loja.

In [49]:
# Unindo o dataset.
train_full = pd.merge(train, items, on='item_id', how='left')
train_full = pd.merge(train_full, items_cat, on='item_category_id', how='left')
train_full = pd.merge(train_full, shops, on='shop_id', how='left')
train_clean = train_full.drop(['date', 'item_name', 'shop_name', 'item_category_name'], axis=1)
train_clean['revenue'] = train_full['item_cnt_day'] * train_full['item_price']

In [59]:
# Agrupando por mês.
df = train_clean.pivot_table(index=['shop_id'], 
                             columns=['date_block_num'],
                             values=['item_cnt_day'],
                             aggfunc='sum', fill_value=0)

df.columns = df.columns.droplevel(0)

# Array com o número de meses:
months = train_clean['date_block_num'].nunique()

# Datas para serem colocadas no eixo x:
date_range = pd.date_range(start='1/2013', periods=months, freq='M')
date_range = [ str(date_range.year[month]) + '/' + str(date_range.month_name()[month]) for month in range(months)]

In [116]:
# Loja 37 é uma boa escolha.
shop = 37

items_sold = df.loc[shop, :]

In [152]:
trace1 = go.Scatter(
    x=date_range,
    y=items_sold,
    name='shop ' + str(shop),
)
fig = tools.make_subplots(rows=1, cols=1)

fig.append_trace(trace1, 1, 1)

fig['layout'].update(height=500, width=900, title='Nº de itens vendidos por mês',
                     xaxis = dict())
iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [445]:
# Função para fazer o teste e mostrar os resultados.

def adf_teste(time_series, regression, maxlag=0):
    #Fazendo o teste.
    adf_test = adfuller(items_sold, regression=regression, maxlag=maxlag, autolag=None)

    # Arrumando as variáveis para o print.
    adf_output = pd.Series(adf_test[0:4], index=['Test Statistic','p-value',
                                                 '#Lags Used',
                                                 'Number of Observations Used'])
    for percent, value in dftest[4].items():
        
        adf_output['Critical Value (%s)' % percent] = value
        
    print('Resultados ADF: \n')
    print(adf_output)

In [456]:
adf_teste(osc_0, 'ct', maxlag=0)

Resultados ADF: 

Test Statistic                 -3.400633
p-value                         0.051303
#Lags Used                      0.000000
Number of Observations Used    33.000000
Critical Value (1%)            -3.752928
Critical Value (5%)            -2.998500
Critical Value (10%)           -2.638967
dtype: float64


In [462]:
#define function for kpss test
from statsmodels.tsa.stattools import kpss
#define KPSS
def kpss_test(timeseries):
    print ('Results of KPSS Test:')
    kpsstest = kpss(timeseries, regression='c')
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
    for key,value in kpsstest[3].items():
        kpss_output['Critical Value (%s)'%key] = value
    print(kpss_output)

In [468]:
kpss_output()

Results of KPSS Test:
Test Statistic            1.93871
p-value                   0.01000
Lags Used                17.00000
Critical Value (10%)      0.34700
Critical Value (5%)       0.46300
Critical Value (2.5%)     0.57400
Critical Value (1%)       0.73900
dtype: float64


/home/ronaldo/anaconda3/envs/Data_Science/lib/python3.6/site-packages/statsmodels/tsa/stattools.py:1276: InterpolationWarning:

p-value is smaller than the indicated p-value



#### ADF (Augmented Dickey Fuller) Test

In [312]:
# Fazendo o teste.
adf_test = adfuller(items_sold, regression='c', maxlag=0, autolag='AIC')

# Arrumando as variáveis para o print.
adf_output = pd.Series(adf_test[0:4], index=['Test Statistic','p-value',
                                             '#Lags Used',
                                             'Number of Observations Used'])
for percent, value in dftest[4].items():
    
    adf_output['Critical Value (%s)' % percent] = value
        
print('Resultados ADF: \n')
print(adf_output)

Resultados ADF: 

Test Statistic                 -2.658821
p-value                         0.081439
#Lags Used                      0.000000
Number of Observations Used    33.000000
Critical Value (1%)            -3.752928
Critical Value (5%)            -2.998500
Critical Value (10%)           -2.638967
dtype: float64


Perceba que essa série não é estacionária, é fácil de ver a tendência de queda nas vendas e a variância (minímo e máximo) assumindo valores diferentes no tempo. 

### Transformando em uma série estacionária

### Separando o dataset

### Considerações finais

# Série temporal

   - [Random Walk](#Random-walk)
   - [ARIMA](#ARIMA)
   - [SARIMA](#SARIMA)
   - [Considerações finais](#Considerações-finais)

### Random walk

### ARIMA

### SARIMA

### Considerações finais

# Apéndice

### O que é a hipótese nula e como é utilizado no ADF teste

Show off:

1. Explicar o teste ADF de verdade:
    
    - Explicar de forma geral o que é a hipótese nula e dar o exemplo dos dados.
    - Explicitar como o p-value ajuda na inferência da hipótese.
    - Começar assumindo o random walk, sem dar muito explicação.
    - Falar da raíz unitária, sem dar muita explicação.
    - Aplicar ao teste ADF a Null hypothesis e definir qual é a nossa hipótese.
    - Explicar o p-value.

- Null hypothesis ($H_0$): Existe uma raiz unitária, indicando que a série não é estacionária.
- Alternate Hypothesis ($H_1$): Não existe uma raiz unitária, indicando que a série é estacionária.

>Se minha hipótese $H_0$ é falsa então $H_1$ é obrigatoriamente verdadeira.

**p-value** é a probabilidade do evento ocorrer.

- Se p-value < 5% o evento é pouco provável, ou seja, podemos considerar que não ocorre.

O que queremos mostrar é que, olhando para os dados que temos, $H_0$ é pouco provável que aconteça, logo $H_1$ é acotence.

livro https://www.stata.com/manuals13/tsdfuller.pdf

https://en.wikipedia.org/wiki/P-value#cite_note-7
The p-value is used in the context of null hypothesis testing in order to quantify the idea of statistical significance of evidence.[note 1] Null hypothesis testing is a reductio ad absurdum argument adapted to statistics. In essence, a claim is assumed valid if its counter-claim is improbable.

As such, the only hypothesis that needs to be specified in this test and which embodies the counter-claim is referred to as the null hypothesis (that is, the hypothesis to be nullified). A result is said to be statistically significant if it allows us to reject the null hypothesis. That is, as per the reductio ad absurdum reasoning, the statistically significant result should be highly improbable if the null hypothesis is assumed to be true. The rejection of the null hypothesis implies that the correct hypothesis lies in the logical complement of the null hypothesis. However, unless there is a single alternative to the null hypothesis, the rejection of null hypothesis does not tell us which of the alternatives might be the correct one.


The smaller the p-value, the higher the significance because it tells the investigator that the hypothesis under consideration may not adequately explain the observation. The null hypothesis {\displaystyle H} H is rejected if any of these probabilities is less than or equal to a small, fixed but arbitrarily pre-defined threshold value {\displaystyle \alpha } \alpha , which is referred to as the level of significance.

**One roll of a pair of dice**
Suppose a researcher rolls a pair of dice once and assumes a null hypothesis that the dice are fair, not loaded or weighted toward any specific number/roll/result; uniform. The test statistic is "the sum of the rolled numbers" and is one-tailed. The researcher rolls the dice and observes that both dice show 6, yielding a test statistic of 12. The p-value of this outcome is 1/36 (because under the assumption of the null hypothesis, the test statistic is uniformly distributed) or about 0.028 (the highest test statistic out of 6×6 = 36 possible outcomes). If the researcher assumed a significance level of 0.05, this result would be deemed significant and the hypothesis that the dice are fair would be rejected.

In this case, a single roll provides a very weak basis (that is, insufficient data) to draw a meaningful conclusion about the dice. This illustrates the danger with blindly applying p-value without considering the experiment design.

### O andar do bêbado

Explicar o random walk

https://www.stata.com/manuals13/tsdfuller.pdf
- Pure Random Walk (Yt = Yt-1 + εt ) Random walk predicts that the value at time "t" will be equal to the last period value plus a stochastic (non-systematic) component that is a white noise, which means εt is independent and identically distributed with mean "0" and variance "σ²." Random walk can also be named a process integrated of some order, a process with a unit root or a process with a stochastic trend. It is a non-mean-reverting process that can move away from the mean either in a positive or negative direction. Another characteristic of a random walk is that the variance evolves over time and goes to infinity as time goes to infinity; therefore, a random walk cannot be predicted.
- Random Walk with Drift (Yt = α + Yt-1 + εt ) If the random walk model predicts that the value at time "t" will equal the last period's value plus a constant, or drift (α), and a white noise term (εt), then the process is random walk with a drift. It also does not revert to a long-run mean and has variance dependent on time.
- Deterministic Trend (Yt = α + βt + εt ) Often a random walk with a drift is confused for a deterministic trend. Both include a drift and a white noise component, but the value at time "t" in the case of a random walk is regressed on the last period's value (Yt-1), while in the case of a deterministic trend it is regressed on a time trend (βt). A non-stationary process with a deterministic trend has a mean that grows around a fixed trend, which is constant and independent of time.
- Random Walk with Drift and Deterministic Trend (Yt = α + Yt-1 + βt + εt ) Another example is a non-stationary process that combines a random walk with a drift component (α) and a deterministic trend (βt). It specifies the value at time "t" by the last period's value, a drift, a trend and a stochastic component. (To learn more about random walks and trends, see our Financial Concepts tutorial.)

# Referências

- O material parece bem embasado e tem um carater mais técnico: [QuantStart](https://www.quantstart.com/articles#time-series-analysis)
- Mais um material técnico: [Fuqua School of Business](http://people.duke.edu/~rnau/411home.htm)
- Passo a passo para time series forecasting: [Comprehensive guide to creating time series forecast](https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/) <- Esse site parece bom.
- [Univariate versus Multivariate](https://www.analyticsvidhya.com/blog/2018/09/multivariate-time-series-guide-forecasting-modeling-python-codes/)
- [Co2 Emission Forecast ARIMA](https://www.kaggle.com/berhag/co2-emission-forecast-with-python-seasonal-arima)
- [Climate Change Forecast SARIMA](https://www.kaggle.com/leandrovrabelo/climate-change-forecast-sarima-model?utm_medium=email&utm_source=intercom&utm_campaign=datanotes-2019)
- Outro site mais técnico: [Forecasting: Principles and Practice](https://otexts.com/fpp2/non-seasonal-arima.html)
- 2[A Gentle Introduction to Handling a Non-Stationary Time Series in Python](https://www.analyticsvidhya.com/blog/2018/09/non-stationary-time-series-python/)
- [1](https://www.analyticsvidhya.com/blog/2018/10/predicting-stock-price-machine-learningnd-deep-learning-techniques-python/) <- Knn